In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import numpy as np

import GPyOpt

import argparse

from utils import tf_config, preprocess_data, search_algorithms, train, losses_utils, metrics

from models import fmri_ae, eeg_to_fmri, uniconv_fmri

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os

from sklearn.model_selection import train_test_split, KFold

import time

dataset="01"
memory_limit=1500
n_individuals=10
interval_eeg=10

tf_config.set_seed(seed=42)
tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit)

with tf.device('/CPU:0'):
    train_data, _ = preprocess_data.dataset(dataset, n_individuals=n_individuals,
                                            interval_eeg=interval_eeg, 
                                            ind_volume_fit=False,
                                            standardize_fmri=True,
                                            iqr=False,
                                            verbose=True)
    eeg_train, fmri_train =train_data

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy
I: Starting to Load Data
I: Finished Loading Data
I: Pairs Created


In [2]:
eeg_train = eeg_train[:100]
fmri_train = fmri_train[:100]

## Build fMRI AE

In [3]:
theta = (0.002980911194116198, 0.0004396489214334123, (9, 9, 4), (1, 1, 1), 4, (7, 7, 7), 4, True, True, True, True, 3, 1)

In [4]:
#unroll hyperparameters
learning_rate=float(theta[0])
weight_decay = float(theta[1])
kernel_size = theta[2]
stride_size = theta[3]
batch_size=int(theta[4])
latent_dimension=theta[5]
n_channels=int(theta[6])
max_pool=bool(theta[7])
batch_norm=bool(theta[8])
skip_connections=bool(theta[9])
dropout=bool(theta[10])
n_stacks=int(theta[11])
outfilter=int(theta[12])
local=True

In [10]:
na_specification = ([(10,20,2),(10,20,2)], 
                    [(1,1,1),(1,1,1)],
                   True,
                   (2,2,1),
                   (1,1,1))

In [16]:
def block18(x, operation, kernel_size, stride_size, n_channels,
            maxpool=True, batch_norm=True, weight_decay=0.000,  padding="valid",
            maxpool_k=None, maxpool_s=None,
            seed=None):

    x = operation(filters=n_channels, kernel_size=kernel_size, strides=stride_size,
                    kernel_regularizer=tf.keras.regularizers.L2(weight_decay),
                    bias_regularizer=tf.keras.regularizers.L2(weight_decay),
                    kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed),
                    padding=padding)(x)
    if(maxpool):
        x = tf.keras.layers.MaxPool3D(pool_size=maxpool_k, strides=maxpool_s)(x)
    if(batch_norm):
        x = tf.keras.layers.BatchNormalization()(x)

    return tf.keras.layers.ReLU()(x)


def skip_block18(x, skip_x, operation, kernel_size, stride_size, n_channels,
                maxpool=True, batch_norm=True, weight_decay=0.000, padding="valid",
                maxpool_k=None, maxpool_s=None,
                seed=None):

    skip_x = operation(filters=n_channels, kernel_size=kernel_size, strides=stride_size,
                    kernel_regularizer=tf.keras.regularizers.L2(weight_decay),
                    bias_regularizer=tf.keras.regularizers.L2(weight_decay),
                    kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed),
                    padding=padding)(skip_x)

    if(maxpool):
        skip_x = tf.keras.layers.MaxPool3D(pool_size=maxpool_k, strides=maxpool_s)(skip_x)
    if(batch_norm):
        skip_x = tf.keras.layers.BatchNormalization()(skip_x)

    x = tf.keras.layers.Add()([x, skip_x])

    return tf.keras.layers.ReLU()(x)

def stack18(x, previous_block_x, operation, kernel_size, stride_size, n_channels,
                        maxpool=True, batch_norm=True, 
                        weight_decay=0.000, skip_connections=False,
                        maxpool_k=None, maxpool_s=None,
                        seed=None):
    #downsampling block    
    x = block18(x, operation, kernel_size, stride_size, n_channels,
            maxpool=maxpool, batch_norm=batch_norm, 
            maxpool_k=maxpool_k, maxpool_s=maxpool_s,
            weight_decay=weight_decay, padding="valid",
            seed=seed)

    #non downsampling block
    x = block18(x, operation, 3, 1, n_channels,
            maxpool=False, batch_norm=batch_norm, 
            weight_decay=weight_decay, padding="same",
            seed=seed)

    #skip connection
    if(skip_connections):
        x = skip_block18(x, previous_block_x, operation, 
                        kernel_size, stride_size, n_channels,
                        maxpool=maxpool, batch_norm=batch_norm,
                        maxpool_k=maxpool_k, maxpool_s=maxpool_s,
                        weight_decay=weight_decay, padding="valid",
                        seed=seed)

    return x



"""
This class implements an architecture for EEG to fMRI transcription

encode: architecture that encodes the EEG signal to a space where an instance of fMRI is also represented

decode: architecture that maps the encoded representation to the fMRI space representation

call: encode and decode

"""

class EEG_to_fMRI(tf.keras.Model):


    """
        NA_specification - tuple - (list1, list2, bool, tuple1, tuple2)
                                    * list1 - kernel sizes
                                    * list2 - stride sizes
                                    * bool - maxpool
                                    * tuple1 - kernel size of maxpool
                                    * tuple2 - stride size of maxpool
                                    Example:
                                    na = ([(2,2,2), (2,2,2)], [(1,1,1), (1,1,1)], True, (2,2,2), (1,1,1))
                                    na is a neural architecture with 2 layers, kernel of size 2 for all 3 dimensions
                                    stride of size 1 for all dimensions, between each layer a max pooling operation 
                                    is applied with kernel size 2 for all dimensions and stride size 1 for all dimensions

    """
    def __init__(self, latent_shape, input_shape, na_spec, n_channels,
                weight_decay=0.000, skip_connections=False, batch_norm=True,
                dropout=False, local=True, seed=None, fmri_args=None):
        super(EEG_to_fMRI, self).__init__()

        self.fmri_ae = fmri_ae.fMRI_AE(*fmri_args)

        self.build_encoder(latent_shape, input_shape, na_spec, n_channels, 
                            dropout=dropout, weight_decay=weight_decay, 
                            skip_connections=skip_connections, local=local, 
                            batch_norm=batch_norm, seed=seed)
        self.build_decoder()

    def build_encoder(self, latent_shape, input_shape, na_spec, n_channels, 
                            dropout=False, weight_decay=0.000, 
                            skip_connections=False, batch_norm=True, 
                            local=True, seed=None):

        input_shape = tf.keras.layers.Input(shape=input_shape)

        x = input_shape
        previous_block_x = input_shape

        for i in range(len(na_spec[0])):
            #x = fmri_ae.stack(x, previous_block_x, tf.keras.layers.Conv3D, 
            x = stack18(x, previous_block_x, tf.keras.layers.Conv3D, 
                        na_spec[0][i], na_spec[1][i], n_channels,
                        maxpool=na_spec[2], batch_norm=batch_norm, weight_decay=weight_decay, 
                        maxpool_k=na_spec[3], maxpool_s=na_spec[4],
                        skip_connections=skip_connections, seed=seed)
            previous_block_x=x

        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.experimental.RandomFourierFeatures(latent_shape[0]*latent_shape[1]*latent_shape[2],
                                                              trainable=True)(x)

        if(dropout):
            x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.Reshape(latent_shape)(x)

        self.eeg_encoder = tf.keras.Model(input_shape, x)
        self.fmri_encoder = self.fmri_ae.encoder

    def build_decoder(self):
        self.decoder = self.fmri_ae.decoder

    def build(self, input_shape1, input_shape2):
        self.eeg_encoder.build(input_shape=input_shape1)

        self.fmri_ae.build(input_shape=input_shape2)        
        self.fmri_encoder.build(input_shape=input_shape2)

        self.built=True

    def call(self, X, training=True):
        x1, x2 = X

        z1 = self.eeg_encoder(x1)
        z2 = self.fmri_encoder(x2)

        if(training):
            return [self.decoder(z1), z1, z2]
        return self.decoder(z1)

In [17]:
with tf.device('/CPU:0'):
    model = EEG_to_fMRI(latent_dimension, eeg_train.shape[1:], na_specification, 4,
                        weight_decay=0.000, skip_connections=True,
                        batch_norm=True, #dropout=False,
                        local=True, seed=None, 
                        fmri_args = (latent_dimension, fmri_train.shape[1:], 
                        kernel_size, stride_size, n_channels, 
                        max_pool, batch_norm, weight_decay, skip_connections,
                        n_stacks, True, False, outfilter, dropout))
    
    model.build(eeg_train.shape, fmri_train.shape)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_fn = losses_utils.mse_cosine

    train_set = tf.data.Dataset.from_tensor_slices((eeg_train, fmri_train)).batch(batch_size)
    dev_set= tf.data.Dataset.from_tensor_slices((eeg_train, fmri_train)).batch(1)

(None, 55, 55, 27, 4)
(None, 46, 46, 24, 4)
(None, 37, 37, 21, 4)


In [18]:
loss_history = train.train(train_set, model, optimizer, 
                            loss_fn, epochs=1, 
                            u_architecture=True,
                            val_set=dev_set, verbose=True, verbose_batch=True)[0]

Batch ... with loss: 1.5177221
Batch ... with loss: 1.1963927
Batch ... with loss: 1.0261576
Batch ... with loss: 0.9754716
Batch ... with loss: 0.96071905


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-ce2b11b6d8d2>", line 1, in <module>
    loss_history = train.train(train_set, model, optimizer,
  File "../utils/train.py", line 114, in train
    batch_loss = train_step(model, batch_set, opt, loss_fn, u_architecture=u_architecture).numpy()
  File "../utils/train.py", line 25, in train_step
    return apply_gradient(model, optimizer, loss_fn, x, x[1])
  File "../utils/train.py", line 13, in apply_gradient
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py", line 631, in apply_gradients
    return distribute_ctx.get_replica_context().merge_call(
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.

TypeError: object of type 'NoneType' has no len()

## Prediction vs ground truth plot

In [7]:
from utils import viz_utils

save_path = "/home/david/eeg_to_fmri/plots/100_instances/"
plot_format="png"

instance = 1
for instance_x, instance_y in dev_set.repeat(1):
    fig = viz_utils.plot_3D_representation_projected_slices(instance_y.numpy()[0])
    plt.savefig(save_path + str(instance) + "_ground_truth."+plot_format, format=plot_format)
    
    fig = viz_utils.plot_3D_representation_projected_slices(model([instance_x, instance_y])[0].numpy()[0])
    plt.savefig(save_path + str(instance) + "_predicted."+plot_format, format=plot_format)
    
    instance += 1

ERROR! Session/line number was not unique in database. History logging moved to new session 466


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-64bcefe70a14>", line 7, in <module>
    for instance_x, instance_y in dev_set.repeat(1):
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 422, in __iter__
    return iterator_ops.OwnedIterator(self)
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 682, in __init__
    self._create_iterator(dataset)
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 686, in _create_iterator
    dataset = dataset._apply_options()
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 397,

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-64bcefe70a14>", line 7, in <module>
    for instance_x, instance_y in dev_set.repeat(1):
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 422, in __iter__
    return iterator_ops.OwnedIterator(self)
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 682, in __init__
    self._create_iterator(dataset)
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 686, in _create_iterator
    dataset = dataset._apply_options()
  File "/home/david/anaconda3/envs/eeg_fmri/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 397,

TypeError: object of type 'NoneType' has no len()

## Structural Similarity Index

In [ ]:
metrics.ssim(dev_set, model)